<a href="https://colab.research.google.com/github/hfarazul/AutoJob/blob/main/AutoJob_Resume_And_Prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Crew to update resume based on job description, and intervieew prep guidelines**

Multi-agent system.

In [ ]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

# Warning control
import warnings
warnings.filterwarnings('ignore')

from crewai import Agent, Task, Crew

import os

# Directly set your OpenAI and Serper API keys here
openai_api_key = "your-api-key"
serper_api_key = "your-api-key" # you can find it here https://serper.dev/api-key, don't worry, it has enough free credits.

# Set the environment variables
os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4-turbo'
os.environ["SERPER_API_KEY"] = serper_api_key

## crewAI Tools

In [26]:
import os
from google.colab import files

# Upload a file from your device
uploaded = files.upload()

# Get the file name
file_path = list(uploaded.keys())[0]
print(f"Uploaded file: {file_path}")

# Display the contents of the file
with open(file_path, 'r') as f:
    content = f.read()
    print(content)

# Check the file extension and process accordingly
file_extension = os.path.splitext(file_path)[1].lower()

# Use the uploaded file path
resume_file_path = file_path

# Initialize the tools with the correct file path
from crewai_tools import (
    FileReadTool,
    ScrapeWebsiteTool,
    SerperDevTool,
    MDXSearchTool
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path=resume_file_path)
semantic_search_resume = MDXSearchTool(mdx=resume_file_path)

Saving Sanskar_Resume.md to Sanskar_Resume (2).md
Uploaded file: Sanskar_Resume (2).md
+-----------------+-----------------+-----------------+-----------------+
| > �             | > Sanskar       |                 | > �sansk        |
| > (             | > Agrawal       |                 | ar107.github.io |
| +91)-7478050999 |                 |                 |                 |
+=================+=================+=================+=================+
|                 | > �sanskaragraw | > �github       |                 |
|                 | al107@gmail.com | .com/sanskar107 |                 |
+-----------------+-----------------+-----------------+-----------------+

> **ABOUT ME**

I am a Machine Learning Researcher and Engineer based in India with 4
years of experience. I specialize in 3D Computer vision and Machine
Learning, with a keen interest in tackling hard technical problems
through R&D.

> **EDUCATION**

+-----------------------------------+-------------------------------

Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  2.49it/s]


## Creating Agents, Tasks and the Crew

In [27]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    )
)

# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    )
)

# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    )
)

# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    )
)





# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)


# Creating the crew
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=True
)

# **Make the neccessary changes here and Run the crew**

*   Job Posting URL
*   Your Github URL(if you have one)
*   A small writeup of your career; see the example (it does have to be fancy, just give the agents some pointers to focus on)



In [29]:
job_application_inputs = {
    'job_posting_url': 'https://www.linkedin.com/jobs/view/machine-learning-engineer-applied-ai-at-openai-3915648739/',
    'github_url': 'https://github.com/sanskar107/',
    'personal_writeup': """Sanskar Agrawal is a seasoned Machine Learning Researcher and Engineer with four years of experience, specializing in 3D Computer Vision and Machine Learning. He graduated with a Bachelor of Technology in Electrical Engineering from the Indian Institute of Technology, Kharagpur, and has a minor in Computer Science and Engineering. He currently leads a team at Preimage.ai, where he has contributed to significant advancements in computer vision models and 3D reconstruction techniques. Previously, he worked at Intel Intelligent Systems Lab, focusing on 3D semantic segmentation and scene relighting. Sanskar has published several research papers in reputable conferences and journals and has been recognized for his achievements in various competitions, including the Inter IIT Technology Meet and the Intelligent Ground Vehicle Competition. His technical expertise includes proficiency in C++, Python, PyTorch, TensorFlow, and CUDA."""
}

### this execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

 [DEBUG]: == Working Agent: Tech Job Researcher
 [INFO]: == Starting Task: Analyze the job posting URL provided (https://www.linkedin.com/jobs/view/machine-learning-engineer-applied-ai-at-openai-3915648739/) to extract key skills, experiences, and qualifications required. Use the tools to gather content and identify and categorize the requirements.
 [DEBUG]: == [Tech Job Researcher] Task output: 


 [DEBUG]: == Working Agent: Personal Profiler for Engineers
 [INFO]: == Starting Task: Compile a detailed personal and professional profile using the GitHub (https://github.com/sanskar107/) URLs, and personal write-up (Sanskar Agrawal is a seasoned Machine Learning Researcher and Engineer with four years of experience, specializing in 3D Computer Vision and Machine Learning. He graduated with a Bachelor of Technology in Electrical Engineering from the Indian Institute of Technology, Kharagpur, and has a minor in Computer Science and Engineering. He currently leads a team at Preimage.ai, wher

```markdown
> **ABOUT ME**

I am a Machine Learning Researcher and Engineer based in India with 4
years of experience. My expertise lies in deploying scalable machine learning models, particularly in computer vision, using frameworks like PyTorch and TensorFlow. I have a proven track record of leading projects from concept to implementation, focusing on innovative solutions in AI.

> **EDUCATION**

+-----------------------------------+-----------------------------------+
| > **Indian Institute of           | > **July, 2020**\                 |
| > Technology, Kharagpur**\        | > *Kharagpur, India*              |
| > *Bachelor of Technology in      |                                   |
| > Electrical Engineering; GPA:    |                                   |
| > 8.7/10;*                        |                                   |
+===================================+===================================+
+-----------------------------------+-----------------------------------+

> *•* **Minor**: Computer Science and Engineering *GPA: 9.1/10*

> **PUBLICATION**

> **\[1\] Objects with Lighting: A Real-World Dataset for Evaluating
> Object Relighting**\
> *Benjamin Ummenhofer, **Sanskar Agrawal**, Yixing Lao, Kai Zhang,
> Stephan R. Richter* *3DV 2024*

> **\[2\] Prior Guided GAN Based Semantic Inpainting**\
> *Avisek Lahiri, **Sanskar Agrawal**, Arnav Jain, Pabitra Mitra, Prabir
> Biswas* *CVPR 2020*

> **\[3\] Off-Road Lane Detection Using Superpixel Clustering And RANSAC
> Curve Fitting**\
> ***Sanskar Agrawal**, Indu Kant Deo, Siddhant Haldar, G Rahul Kranti
> Kiran* *ICARCV 2018*

> **\[4\] Design and Implementation of Autonomous Ground Vehicle for
> constrained environments** *G Rahul Kranti Kiran, **Sanskar Agrawal**,
> Indu Kant Deo, Siddhant Haldar, Het Shah* *IEEE IRC*

> **WORK EXPERIENCE**

+-----------------------------------+-----------------------------------+
| > **Preimage.ai**\                | > **Bengaluru, India**\           |
| > *Machine Learning Lead*         | > *Oct 2022 -- Present*           |
+===================================+===================================+

> *•* **Lead and innovator** in deploying advanced machine learning models, focusing on deep learning and transformers. Developed scalable data pipelines and optimized machine learning models for performance and accuracy, ensuring they are production-ready.

> *•* **Depth Estimation with MVS** - Enhanced depth estimation techniques using Vision Transformers, significantly improving the accuracy of multi-view stereo images.

> *•* **3D Reconstruction with NeRFs** - Innovated in neural radiance fields for 3D modeling, enhancing the scalability and performance using custom CUDA kernels for optimizations.

> *•* **PointCloud Fusion** - Developed a robust multi-view fusion pipeline, utilizing advanced GPU techniques to achieve significant performance enhancements.

> *•* Currently leading projects on real-time rendering for AI-driven applications, directly contributing to innovative user experiences.

> **Intel Intelligent Systems Lab(ISL)** **Bengaluru, India** *Research
> Engineer* *Jul 2020 - Oct 2022*

> *•* Spearheaded the development of new algorithms for 3D semantic segmentation and scene relighting, contributing to state-of-the-art advances in physically based rendering.

> **RESEARCH EXPERIENCE**

> **Eye Gaze Estimation** *\|* **Bachelor's Thesis** *Prof. Pabitra
> Mitra*

May 2019 - Apr 2020

> *•* Pioneered an unsupervised approach to Eye Gaze Estimation, utilizing advanced machine learning techniques to set new benchmarks in the field.

> **Image Inpainting** Apr 2019 - Dec 2019

> *Prof. Pabitra Mitra*

> *•* Led the development of a novel GAN-based approach for Image Inpainting, significantly advancing the state-of-the-art in generative modeling.

> **TECHNICAL SKILLS**

> *•* **Languages:** C++, Python, Bash, Julia, Matlab

> *•* **Tools / Frameworks:** PyTorch, TensorFlow, CUDA, Docker, OpenMP, Git.

> **COURSEWORK**

> *•* **University:** Reinforcement Learning, Algorithms, Advanced
> Computer Vision, High Performance Parallel Programming, Deep Learning,
> Image Processing, Machine Learning.

> *•* **Online:** Introduction to Computer Vision, Artificial
> Intelligence for Robotics, Deep Learning Specialization, CS231n,
> Natural Language Processing.

> **ACHIEVEMENTS**

> *•* Consistently recognized for top performance in autonomous navigation and machine learning competitions, demonstrating advanced technical proficiency and innovative problem-solving skills.
```

- Dislplay the generated `interview_materials.md` file.

In [31]:
from IPython.display import Markdown, display
display(Markdown("./tailored_resume.md"))

```markdown
> **ABOUT ME**

I am a Machine Learning Researcher and Engineer based in India with 4
years of experience. My expertise lies in deploying scalable machine learning models, particularly in computer vision, using frameworks like PyTorch and TensorFlow. I have a proven track record of leading projects from concept to implementation, focusing on innovative solutions in AI.

> **EDUCATION**

+-----------------------------------+-----------------------------------+
| > **Indian Institute of           | > **July, 2020**\                 |
| > Technology, Kharagpur**\        | > *Kharagpur, India*              |
| > *Bachelor of Technology in      |                                   |
| > Electrical Engineering; GPA:    |                                   |
| > 8.7/10;*                        |                                   |
+===================================+===================================+
+-----------------------------------+-----------------------------------+

> *•* **Minor**: Computer Science and Engineering *GPA: 9.1/10*

> **PUBLICATION**

> **\[1\] Objects with Lighting: A Real-World Dataset for Evaluating
> Object Relighting**\
> *Benjamin Ummenhofer, **Sanskar Agrawal**, Yixing Lao, Kai Zhang,
> Stephan R. Richter* *3DV 2024*

> **\[2\] Prior Guided GAN Based Semantic Inpainting**\
> *Avisek Lahiri, **Sanskar Agrawal**, Arnav Jain, Pabitra Mitra, Prabir
> Biswas* *CVPR 2020*

> **\[3\] Off-Road Lane Detection Using Superpixel Clustering And RANSAC
> Curve Fitting**\
> ***Sanskar Agrawal**, Indu Kant Deo, Siddhant Haldar, G Rahul Kranti
> Kiran* *ICARCV 2018*

> **\[4\] Design and Implementation of Autonomous Ground Vehicle for
> constrained environments** *G Rahul Kranti Kiran, **Sanskar Agrawal**,
> Indu Kant Deo, Siddhant Haldar, Het Shah* *IEEE IRC*

> **WORK EXPERIENCE**

+-----------------------------------+-----------------------------------+
| > **Preimage.ai**\                | > **Bengaluru, India**\           |
| > *Machine Learning Lead*         | > *Oct 2022 -- Present*           |
+===================================+===================================+

> *•* **Lead and innovator** in deploying advanced machine learning models, focusing on deep learning and transformers. Developed scalable data pipelines and optimized machine learning models for performance and accuracy, ensuring they are production-ready.

> *•* **Depth Estimation with MVS** - Enhanced depth estimation techniques using Vision Transformers, significantly improving the accuracy of multi-view stereo images.

> *•* **3D Reconstruction with NeRFs** - Innovated in neural radiance fields for 3D modeling, enhancing the scalability and performance using custom CUDA kernels for optimizations.

> *•* **PointCloud Fusion** - Developed a robust multi-view fusion pipeline, utilizing advanced GPU techniques to achieve significant performance enhancements.

> *•* Currently leading projects on real-time rendering for AI-driven applications, directly contributing to innovative user experiences.

> **Intel Intelligent Systems Lab(ISL)** **Bengaluru, India** *Research
> Engineer* *Jul 2020 - Oct 2022*

> *•* Spearheaded the development of new algorithms for 3D semantic segmentation and scene relighting, contributing to state-of-the-art advances in physically based rendering.

> **RESEARCH EXPERIENCE**

> **Eye Gaze Estimation** *\|* **Bachelor's Thesis** *Prof. Pabitra
> Mitra*

May 2019 - Apr 2020

> *•* Pioneered an unsupervised approach to Eye Gaze Estimation, utilizing advanced machine learning techniques to set new benchmarks in the field.

> **Image Inpainting** Apr 2019 - Dec 2019

> *Prof. Pabitra Mitra*

> *•* Led the development of a novel GAN-based approach for Image Inpainting, significantly advancing the state-of-the-art in generative modeling.

> **TECHNICAL SKILLS**

> *•* **Languages:** C++, Python, Bash, Julia, Matlab

> *•* **Tools / Frameworks:** PyTorch, TensorFlow, CUDA, Docker, OpenMP, Git.

> **COURSEWORK**

> *•* **University:** Reinforcement Learning, Algorithms, Advanced
> Computer Vision, High Performance Parallel Programming, Deep Learning,
> Image Processing, Machine Learning.

> *•* **Online:** Introduction to Computer Vision, Artificial
> Intelligence for Robotics, Deep Learning Specialization, CS231n,
> Natural Language Processing.

> **ACHIEVEMENTS**

> *•* Consistently recognized for top performance in autonomous navigation and machine learning competitions, demonstrating advanced technical proficiency and innovative problem-solving skills.
```

In [30]:
display(Markdown("./interview_materials.md"))

**Interview Questions and Talking Points for Sanskar Agrawal:**

1. **Educational Background and Theoretical Knowledge:**
   - **Question:** "Can you discuss how your educational background and coursework at IIT Kharagpur have prepared you for a role that involves both research and application of machine learning techniques?"
   - **Talking Point:** Emphasize the strong foundation in computer science and electrical engineering, highlighting specific relevant courses such as Advanced Computer Vision and Deep Learning.

2. **Technical Skills and Frameworks:**
   - **Question:** "You've mentioned proficiency in PyTorch and TensorFlow. Could you provide examples of projects where you utilized these frameworks for developing and deploying machine learning models?"
   - **Talking Point:** Discuss the Depth Estimation with MVS project and the 3D Reconstruction with NeRFs, focusing on the use of these frameworks and the technical challenges overcome.

3. **Professional Experience with Machine Learning Models:**
   - **Question:** "In your role at Preimage.ai, you led the development of advanced ML models. How did you ensure these models were scalable and production-ready?"
   - **Talking Point:** Highlight the leadership in deploying scalable models and the specific techniques used to enhance performance and accuracy.

4. **Research Contributions and Publications:**
   - **Question:** "Your publication record is quite impressive. Could you elaborate on your research in 'Prior Guided GAN Based Semantic Inpainting' and its relevance to real-world applications?"
   - **Talking Point:** Focus on the innovative aspects of the research and how it pushes the boundaries of what's achievable with GANs in image processing.

5. **Experience with Large Language Models (LLMs) and Relevant Technologies:**
   - **Question:** "Given the role's emphasis on LLMs, can you discuss any experience you have with training or fine-tuning such models, particularly any challenges you faced and how you addressed them?"
   - **Talking Point:** Reference any relevant projects, even if indirectly related, such as work on neural networks or deep learning models that demonstrate a strong understanding of underlying principles applicable to LLMs.

6. **Leadership and Team Collaboration:**
   - **Question:** "Leading a team at Preimage.ai must have posed various challenges, especially in a cutting-edge field like AI. How did you manage team dynamics and project deadlines?"
   - **Talking Point:** Illustrate leadership style, decision-making processes, and methods for fostering innovation and accountability within teams.

7. **Adaptability and Learning:**
   - **Question:** "The field of AI is rapidly evolving. How do you stay updated with the latest developments and integrate new knowledge into your work?"
   - **TalkingTalk about learning strategies, participation in conferences, or contributions to open-source projects.

8. **Problem-solving and Innovation:**
   - **Question:** "Can you describe a particularly challenging problem you solved in your previous roles and the innovative approach you used?"
   - **Talking Point:** Discuss specific examples that showcase problem-solving skills and creativity, such as the development of new algorithms for 3D semantic segmentation or innovations in 3D modeling.

These questions and talking points are designed to help Sanskar Agrawal effectively demonstrate his suitability for the Machine Learning Engineer position at OpenAI, focusing on his strengths, achievements, and the potential to contribute to innovative projects.